In [ ]:
import os

In [ ]:
os.environ["OPENAI_API_KEY"]
# os.environ['CHROMA_API_IMPL']='rest'
# os.environ['CHROMA_SERVER_CORS_ALLOW_ORIGINS']=''

In [ ]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org  chromadb langchain openai tiktoken langchain_openai -q

In [ ]:
!pip install sentence-transformers -q

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import  DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.llms import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
# load multiple document and process documants
loader = DirectoryLoader("./bbc/sport", glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
# split the text into smaler chunks
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_spliter.split_documents(documents)

In [ ]:
# Create a ChromaDB
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(host="chroma", port = 8000, settings=Settings(allow_reset=True, anonymized_telemetry=False))

In [ ]:
persist_directory = "db"
# embedding = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [ ]:
vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embedding,
    persist_directory = persist_directory
)

In [ ]:
#persist the db to the disk
vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory = persist_directory, embedding_function = embedding)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain.vectorstores import FAISS
from langchain.schema.vectorstore import VectorStoreRetriever
retriever2 = VectorStoreRetriever(vectorstore=vectordb)

In [ ]:
retriever2.search_type

In [ ]:
retriever2.search_kwargs

In [ ]:
from langchain_openai import ChatOpenAI
turbo_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = turbo_llm,
    chain_type="stuff",
    retriever = retriever2,
    # return_source_documents=True
)

In [ ]:
query = "What is the news about soccer"
llm_response = qa_chain.invoke(query)
llm_resp0nse